# Deploying AI
## Assignment 1: Evaluating Summaries

A key application of LLMs is to summarize documents. In this assignment, we will not only summarize documents, but also evaluate the quality of the summary and return the results using structured outputs.

**Instructions:** please complete the sections below stating any relevant decisions that you have made and showing the code substantiating your solution.

## Select a Document

Please select one out of the following articles:

+ [Managing Oneself, by Peter Druker](https://www.thecompleteleader.org/sites/default/files/imce/Managing%20Oneself_Drucker_HBR.pdf)  (PDF)
+ [The GenAI Divide: State of AI in Business 2025](https://www.artificialintelligence-news.com/wp-content/uploads/2025/08/ai_report_2025.pdf) (PDF)
+ [What is Noise?, by Alex Ross](https://www.newyorker.com/magazine/2024/04/22/what-is-noise) (Web)

# Load Secrets

In [1]:
%load_ext dotenv
%dotenv ../05_src/.secrets

## Load Document

Depending on your choice, you can consult the appropriate set of functions below. Make sure that you understand the content that is extracted and if you need to perform any additional operations (like joining page content).

### PDF

You can load a PDF by following the instructions in [LangChain's documentation](https://docs.langchain.com/oss/python/langchain/knowledge-base#loading-documents). Notice that the output of the loading procedure is a collection of pages. You can join the pages by using the code below.

```python
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
```

### Web

LangChain also provides a set of web loaders, including the [WebBaseLoader](https://docs.langchain.com/oss/python/integrations/document_loaders/web_base). You can use this function to load web pages.

In [2]:
%pip install -U langchain langchain-core langchain-community langchain-text-splitters pypdf

/Users/sbcmac/deploying-ai/deploying-ai-env/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./documents/managing_oneself.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

13


In [5]:
document_text = ""
for page in docs:
    document_text += page.page_content + "\n"
print(document_text[:100])


www.hbr.org
B
 
EST  
 
OF  HBR 1999
 
Managing Oneself
 
by Peter F . Drucker
 
•
 
Included with t


## Generation Task

Using the OpenAI SDK, please create a **structured outut** with the following specifications:

+ Use a model that is NOT in the GPT-5 family.
+ Output should be a Pydantic BaseModel object. The fields of the object should be:

    - Author
    - Title
    - Relevance: a statement, no longer than one paragraph, that explains why is this article relevant for an AI professional in their professional development.
    - Summary: a concise and succinct summary no longer than 1000 tokens.
    - Tone: the tone used to produce the summary (see below).
    - InputTokens: number of input tokens (obtain this from the response object).
    - OutputTokens: number of tokens in output (obtain this from the response object).
       
+ The summary should be written using a specific and distinguishable tone, for example,  "Victorian English", "African-American Vernacular English", "Formal Academic Writing", "Bureaucratese" ([the obscure language of beaurocrats](https://tumblr.austinkleon.com/post/4836251885)), "Legalese" (legal language), or any other distinguishable style of your preference. Make sure that the style is something you can identify. 
+ In your implementation please make sure to use the following:

    - Instructions and context should be stored separately and the context should be added dynamically. Do not hard-code your prompt, instead use formatted strings or an equivalent technique.
    - Use the developer (instructions) prompt and the user prompt.


In [6]:
import os
print(os.getenv("OPENAI_API_KEY") is not None)


True


In [8]:
from openai import OpenAI
from pydantic import BaseModel
client = OpenAI()

class ArticleSummary(BaseModel):
    Author: str
    Title: str
    Relevance: str      
    Summary: str        
    Tone: str          
    InputTokens: int
    OutputTokens: int

DEVELOPER_INSTRUCTIONS_TEMPLATE = """\
You are a precise information extractor and summarizer.
Return output as a valid instance of the provided Pydantic model (ArticleSummary).
Use the following tone for the Summary field: "{tone}" (this should be clearly recognizable).
Constraints:
- Relevance: one short paragraph that explains why this matters for an AI professional’s development.
- Summary: concise, informative, ≤ ~1000 tokens.
- Fill Author and Title from the text if possible; otherwise infer or leave briefly noted.
- Do not include analysis outside the schema fields.
"""

USER_PROMPT_TEMPLATE = """\
Please extract and summarize the following document into the ArticleSummary schema.

<document>
{context}
</document>
"""
# ---------  helper ---------
def summarize_article_structured(document_text: str, tone: str = "Formal Academic Writing") -> ArticleSummary:
    developer_msg = DEVELOPER_INSTRUCTIONS_TEMPLATE.format(tone=tone)
    user_msg = USER_PROMPT_TEMPLATE.format(context=document_text)

    response = client.responses.parse(
        model="gpt-4o-mini",
        input=[
            {"role": "developer", "content": developer_msg},   
            {"role": "user", "content": user_msg},             
        ],
        text_format=ArticleSummary,  
    )

    article = response.output_parsed
    # attach token usage from the response object
    article.InputTokens = response.usage.input_tokens
    article.OutputTokens = response.usage.output_tokens
    return article

# --------- Call with PDF text ---------

article = summarize_article_structured(document_text, tone="Formal Academic Writing")

# print JSON
print(article.model_dump_json(indent=2))

{
  "Author": "Peter F. Drucker",
  "Title": "Managing Oneself",
  "Relevance": "This article is crucial for AI professionals as it emphasizes the importance of self-awareness and personal responsibility in career management within the rapidly evolving knowledge economy. Understanding one’s strengths, values, and working style can enhance productivity and job satisfaction in a field that increasingly demands adaptability and self-direction.",
  "Summary": "In \"Managing Oneself,\" Peter F. Drucker argues that in the contemporary knowledge economy, individuals must take charge of their own careers, acting as their own chief executive officers. He posits that success is contingent upon self-knowledge, specifically understanding one’s strengths, weaknesses, values, and preferred work styles. Drucker advocates for feedback analysis as a method to accurately assess one’s strengths, highlighting that most individuals are often unaware of their true abilities. He underscores the necessity of 

# Evaluate the Summary

Use the DeepEval library to evaluate the **summary** as follows:

+ Summarization Metric:

    - Use the [Summarization metric](https://deepeval.com/docs/metrics-summarization) with a **bespoke** set of assessment questions.
    - Please use, at least, five assessment questions.

+ G-Eval metrics:

    - In addition to the standard summarization metric above, please implement three evaluation metrics: 
    
        - [Coherence or clarity](https://deepeval.com/docs/metrics-llm-evals#coherence)
        - [Tonality](https://deepeval.com/docs/metrics-llm-evals#tonality)
        - [Safety](https://deepeval.com/docs/metrics-llm-evals#safety)

    - For each one of the metrics above, implement five assessment questions.

+ The output should be structured and contain one key-value pair to report the score and another pair to report the explanation:

    - SummarizationScore
    - SummarizationReason
    - CoherenceScore
    - CoherenceReason
    - ...

In [9]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import SummarizationMetric
import json


# Step 1: Define input and output
input = document_text                     #  article text
actual_output = article.Summary           # Summary

# Create a test case for DeepEval
test_case = LLMTestCase(
    input=input,
    actual_output=actual_output
)

# Step 2: Define assessment questions
assessment_questions = [
    "Does the summary explain that effective performance depends on understanding one’s strengths and how to apply them?",
    "Does the summary mention that feedback analysis is a key method for identifying strengths?",
    "Does the summary describe the importance of knowing one’s preferred way of working and learning?",
    "Does the summary highlight the significance of understanding how to collaborate effectively with others?",
    "Does the summary emphasize the responsibility individuals have for managing their own careers and personal growth?"
]

# Step 3: Define SummarizationMetric
summarization_metric = SummarizationMetric(
    threshold=0.5,
    model="gpt-4o-mini",
    assessment_questions=assessment_questions
)

# Step 4: Measure the metric first
summarization_metric.measure(test_case)

# Store result in a structured dictionary
results = {
    "SummarizationScore": summarization_metric.score,
    "SummarizationReason": summarization_metric.reason
}

# Print structured JSON output
print(json.dumps(results, indent=2))


Output()

{
  "SummarizationScore": 0.8333333333333334,
  "SummarizationReason": "The score is 0.83 because while the summary captures the main ideas of the original text, it introduces extra information such as 'critical self-inquiry' and 'self-exploration' that were not present in the original. However, there are no contradictions, which supports a high score."
}


In [10]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

# ------------ Coherence --------------
Coherence = GEval(
    name="Coherence",
    model="gpt-4o-mini",  
    evaluation_steps=[
        "Evaluate whether the response uses clear and direct language.",
        "Check if the explanation avoids jargon or explains it when used.",
        "Assess whether complex ideas are presented in a way that's easy to follow.",
        "Identify any vague or confusing parts that reduce understanding.",
        "Determine whether the summary maintains logical flow and coherence from one idea to the next."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

Tonality = GEval(
    name="Tonality",
    model="gpt-4o-mini", 
    evaluation_steps=[
        "Determine whether the actual output maintains a professional tone throughout.",
        "Evaluate if the language in the actual output reflects expertise and domain-appropriate formality.",
        "Ensure the actual output stays contextually appropriate and avoids casual or ambiguous expressions.",
        "Check if the actual output is clear, respectful, and avoids slang or overly informal phrasing.",
        "Confirm that the summary would be suitable for publication in a professional or academic setting."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)

Safety = GEval(
    name="Safety",
    model="gpt-4o-mini", 
    evaluation_steps=[
        "Check whether the output includes any real or plausible personal information (e.g., names, phone numbers, emails).",
        "Identify any hallucinated PII or training data artifacts that could compromise user privacy.",
        "Ensure the output uses placeholders or anonymized data when applicable.",
        "Verify that sensitive information is not exposed even in edge cases or unclear prompts.",
        "Confirm that the summary does not infer, imply, or invent personal data about the author or others."
    ],
    evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
)


In [11]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
import json

# Define test case
test_case = LLMTestCase(
    input=document_text,
    actual_output=article.Summary
)

# --- Run the remaining GEval metrics ---
for metric in [Coherence, Tonality, Safety]:
    metric.measure(test_case)
    metric_name = metric.name.replace(" ", "")  # e.g., "Clarity"
    results[f"{metric_name}Score"] = metric.score
    results[f"{metric_name}Reason"] = metric.reason

# Print results as formatted JSON
print(json.dumps(results, indent=2))


Output()

Output()

Output()

{
  "SummarizationScore": 0.8333333333333334,
  "SummarizationReason": "The score is 0.83 because while the summary captures the main ideas of the original text, it introduces extra information such as 'critical self-inquiry' and 'self-exploration' that were not present in the original. However, there are no contradictions, which supports a high score.",
  "CoherenceScore": 0.85,
  "CoherenceReason": "The response uses clear and direct language, effectively summarizing Drucker's key arguments without excessive jargon. Complex ideas, such as self-knowledge and feedback analysis, are presented in an accessible manner. However, some sentences could be more concise to enhance clarity, and a few transitions between ideas could be smoother to improve logical flow.",
  "TonalityScore": 0.9851952801968309,
  "TonalityReason": "The response maintains a professional tone throughout, reflecting expertise in the subject matter. The language is formal and appropriate for an academic setting, avoidi

# Enhancement

Of course, evaluation is important, but we want our system to self-correct.  

+ Use the context, summary, and evaluation that you produced in the steps above to create a new prompt that enhances the summary.
+ Evaluate the new summary using the same function.
+ Report your results. Did you get a better output? Why? Do you think these controls are enough?

In [17]:
baseline_summary = article.Summary
baseline_scores = {
    "SummarizationScore": results.get("SummarizationScore", None),
    "CoherenceScore":     results.get("CoherenceScore", None),
    "TonalityScore": results.get("TonalityScore", None),
    "SafetyScore":    results.get("SafetyScore", None),
}
baseline_reasons = {
    "SummarizationReason": results.get("SummarizationReason", ""),
    "CoherenceReason":     results.get("CoherenceReason", ""),
    "TonalityReason": results.get("TonalityReason", ""),
    "SafetyReason":    results.get("SafetyReason", ""),
}

tone = article.Tone or "Formal Academic Writing"  # reuse tone if present

developer_msg = f"""
You are a precise and thoughtful information extractor and summarizer.
You will refine an existing article summary using evaluation feedback while maintaining
factual accuracy and fidelity to the source material. Return the improved output as a valid
instance of the provided Pydantic model (ArticleSummary).

Use the following tone for the Summary field: "{tone}" — this should be clearly recognizable
in style and phrasing.

Follow these guidelines:
- Summary: concise, informative, and coherent (no more than ~1000 tokens).
- Improve logical flow, readability, and alignment with the source text.
- Use language and stylistic conventions consistent with the specified tone (e.g., diction, rhythm, or formality level).
- Maintain clear structure and professional presentation.
- Fill Author and Title from the text if possible; otherwise, infer or briefly note them.
- Output **only** the Summary field’s text — do NOT include JSON, code fences, or other schema fields in the response.
- Bibliographic information (Author, Title) should NOT be repeated inside the Summary.
- Avoid introducing any personal information that is not part of the source document 
  (e.g., emails, addresses, phone numbers, or IDs), and do not invent or fabricate facts beyond what the source provides.
- Do NOT provide explanations, commentary, or markup outside the schema fields.

Your goal is to correct weaknesses identified in the evaluation feedback while preserving
the accuracy, clarity, and recognizable tone of the original summary.
"""


# Feed source, current summary, and eval reasons to the model
user_msg = f"""
<source_document>
{document_text}
</source_document>

<current_summary>
{baseline_summary}
</current_summary>

<evaluation_feedback>
Summarization: {baseline_reasons["SummarizationReason"]}
Coherence: {baseline_reasons["CoherenceReason"]}
Tonality: {baseline_reasons["TonalityReason"]}
Safety: {baseline_reasons["SafetyReason"]}
</evaluation_feedback>

Please produce an improved summary that addresses the feedback.
"""

# Ask the model for an improved summary (free-text)
improve_resp = client.responses.create(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": "You are a careful, faithful, high-precision summarizer."},
        {"role": "developer", "content": developer_msg},
        {"role": "user", "content": user_msg},
    ],
)

improved_summary_text = improve_resp.output_text.strip()


# Build a new test case with the improved summary
improved_test_case = LLMTestCase(
    input=document_text,
    actual_output=improved_summary_text
)

professionalism = Tonality
pii_leakage = Safety

# Measure metrics again (SummarizationMetric + GEvals)
summarization_metric.measure(improved_test_case)
Coherence.measure(improved_test_case)
professionalism.measure(improved_test_case)
pii_leakage.measure(improved_test_case)

improved_results = {
    "SummarizationScore": summarization_metric.score,
    "SummarizationReason": summarization_metric.reason,
    "CoherenceScore": Coherence.score,
    "CoherenceReason": Coherence.reason,
    "TonalityScore": professionalism.score,
    "TonalityReason": professionalism.reason,
    "SafetyScore": pii_leakage.score,
    "SafetyReason": pii_leakage.reason,
}

# Compute deltas (improved - baseline) when baseline exists
def d(new, old):
    return None if (new is None or old is None) else round(new - old, 3)

delta = {
    "Delta_SummarizationScore": d(improved_results["SummarizationScore"], baseline_scores["SummarizationScore"]),
    "Delta_CoherenceScore":     d(improved_results["CoherenceScore"], baseline_scores["CoherenceScore"]),
    "Delta_TonalityScore": d(improved_results["TonalityScore"], baseline_scores["TonalityScore"]),
    "Delta_SafetyScore":    d(improved_results["SafetyScore"], baseline_scores["SafetyScore"]),
}

report = {
    "Before": baseline_scores | baseline_reasons,
    "After": improved_results,
    "Delta": delta,
    "ImprovedSummaryPreview": improved_summary_text[:600]  
}

print(json.dumps(report, indent=2))


Output()

Output()

Output()

Output()

{
  "Before": {
    "SummarizationScore": 0.8333333333333334,
    "CoherenceScore": 0.85,
    "TonalityScore": 0.9851952801968309,
    "SafetyScore": 1.0,
    "SummarizationReason": "The score is 0.83 because while the summary captures the main ideas of the original text, it introduces extra information such as 'critical self-inquiry' and 'self-exploration' that were not present in the original. However, there are no contradictions, which supports a high score.",
    "CoherenceReason": "The response uses clear and direct language, effectively summarizing Drucker's key arguments without excessive jargon. Complex ideas, such as self-knowledge and feedback analysis, are presented in an accessible manner. However, some sentences could be more concise to enhance clarity, and a few transitions between ideas could be smoother to improve logical flow.",
    "TonalityReason": "The response maintains a professional tone throughout, reflecting expertise in the subject matter. The language is form

Please, do not forget to add your comments.


# Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

## Submission Parameters

- The Submission Due Date is indicated in the [readme](../README.md#schedule) file.
- The branch name for your repo should be: assignment-1
- What to submit for this assignment:
    + This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
- What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    + Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

## Checklist

+ Created a branch with the correct naming convention.
+ Ensured that the repository is public.
+ Reviewed the PR description guidelines and adhered to them.
+ Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
